# 🚀 Proyecto ELT con SpaceX API  
## Notebook 01 – Extracción y Carga

En este notebook se realiza la **extracción de datos desde la API pública de SpaceX** y el **almacenamiento en Delta Lake**.  

Se siguen los pasos de la consigna:  
1. Extracción de **2 o más endpoints**.  
2. Uso de al menos un **endpoint dinámico (actualizable)** y otro **estático**.  
3. Guardado en **formato Delta Lake**.  
4. Aplicación de **extracción incremental y full** según corresponda.  


In [1]:
# =========================
# CELDA DE CONFIGURACIÓN INICIAL
# =========================

import sys
from pathlib import Path

# --- RUTA DEL PROYECTO ---
# Detecta automáticamente la raíz del proyecto buscando la carpeta "src"
def find_project_root(marker="src"):
    path = Path().cwd()
    for _ in range(5):  # sube hasta 5 niveles si es necesario
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"No se encontró la carpeta '{marker}' en los niveles superiores.")

project_root = find_project_root()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root detectado en: {project_root}")

# --- IMPORTS DEL PROYECTO ---
from src.extract import fetch_data, filter_incremental
from src.load import save_to_parquet
from src.utils import setup_logger
from src.transform import load_from_parquet

logger = setup_logger()

Project root detectado en: c:\Users\MONSO\OneDrive\Escritorio\Final-DataEngineering


## 🔗 Endpoints seleccionados

- **Dinámicos:**  
  - `launches/latest` → Último lanzamiento.  
  - `launches/upcoming` → Próximos lanzamientos.  

- **Estáticos:**  
  - `rockets` → Información de cohetes.  
  - `dragons` → Información de cápsulas Dragon.  

In [2]:
# Extracción de datos

# FULL extraction de Rockets
logger.info("Extrayendo Rockets (FULL)...")
rockets_df = fetch_data("rockets")
display(rockets_df.head())


print(f"✅ Extracción completa: {len(rockets_df)} rockets extraídos.")

2025-09-06 17:27:57,398 - INFO - Extrayendo Rockets (FULL)...


,payload_weights,flickr_images,name,type,active,stages,boosters,cost_per_launch,success_rate_pct,first_flight,...,engines.number,engines.type,engines.version,engines.layout,engines.engine_loss_max,engines.propellant_1,engines.propellant_2,engines.thrust_to_weight,landing_legs.number,landing_legs.material
0,"[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...","[https://imgur.com/DaCfMsj.jpg, https://imgur....",Falcon 1,rocket,False,2,0,6700000,40,2006-03-24,...,1,merlin,1C,single,0.0,liquid oxygen,RP-1 kerosene,96.0,0,None
1,"[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm1.staticflickr.com/929/2878733830...,Falcon 9,rocket,True,2,0,50000000,98,2010-06-04,...,9,merlin,1D+,octaweb,2.0,liquid oxygen,RP-1 kerosene,180.1,4,carbon fiber
2,"[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm5.staticflickr.com/4599/385838292...,Falcon Heavy,rocket,True,2,2,90000000,100,2018-02-06,...,27,merlin,1D+,octaweb,6.0,liquid oxygen,RP-1 kerosene,180.1,12,carbon fiber
3,"[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://live.staticflickr.com/65535/489541389...,Starship,rocket,False,2,0,7000000,0,2021-12-01,...,37,raptor,,None,NaN,liquid oxygen,liquid methane,107.0,6,stainless steel


✅ Extracción completa: 4 rockets extraídos.


In [ ]:
# Extracción INCREMENTAL de launches
logger.info("Extrayendo Launches (INCREMENTAL)...")
launches_new = fetch_data("upcoming_launches")
display(launches_new.head())

# Cargar histórico desde Bronze
launches_old = load_from_parquet("upcoming_launches", layer="bronze")

# Filtrar solo lo nuevo
launches_inc = filter_incremental(launches_new, launches_old, time_col="date_utc")

if not launches_inc.empty:
    print(f"✅ Extracción completa: {len(launches_inc)} launches nuevos extraídos.")
else:
    logger.info("No hay datos nuevos para upcoming_launches")

2025-09-06 17:27:57,981 - INFO - Extrayendo Launches (INCREMENTAL)...
2025-09-06 17:27:58,740 - INFO - No hay datos nuevos para upcoming_launches


## 💾 Guardado

Se guarda cada DataFrame en formato **Delta Lake**:  
- Los **endpoints dinámicos** (`latest_launch`, `upcoming_launches`) se almacenan con **particiones por fecha (extracción incremental)**.  
- Los **endpoints estáticos** (`rockets`, `dragons`) se guardan en una única ruta (extracción full).  

In [4]:
# Guardado de datos en Parquet

# FULL save de Rockets
save_to_parquet(rockets_df, "rockets", layer="bronze", incremental=False, mode="overwrite")

# INCREMENTAL save de Launches
if not launches_inc.empty:
    save_to_parquet(launches_inc, "upcoming_launches", layer="bronze", incremental=True, mode="append")

logger.info("Todos los datasets fueron guardados correctamente en Parquet.")

2025-09-06 17:27:58,797 - INFO - Todos los datasets fueron guardados correctamente en Parquet.


[INFO] Guardado FULL en C:\Users\MONSO\OneDrive\Escritorio\Final-DataEngineering\data\bronze\rockets\rockets.parquet
